# Задание:

 *    Выбрать свой источник структурированных данных (прогноз погоды, курс акций, счет спортивных мероприятий, или что-нибудь еще);
 *  Научиться извлекать данные из с выбранной веб-страницы;
 *   Научиться генерировать текст по шаблонам.


### Источник  данных и извлечение нужных полей.

В качестве источника данных будем использовать глобальную службу прогнозирования погоды (Global Forecast System (GFS)). Служба предоставляет прогноз погоды на 8 дней с максимальным разрешением в 0.25 градуса. Прогноз распространяется в виде файлов в формате GRIB (GRIdded Binary). Файлы доступны для скачивания:
http://nomads.ncep.noaa.gov/pub/data/nccf/com/gfs/prod/gfs.2017123006/
Прогноз обновляется каждые 6 часов.
Для работы с grib-файлами нужна библиотека pygrib

Файл прогноза погоды на 30 января 2017г. 
 * cd /home/anaconda/var/grib
 * wget http://nomads.ncep.noaa.gov/pub/data/nccf/com/gfs/prod/gfs.2017123006/gfs.t06z.pgrb2.0p25.f000

In [1]:
import pygrib

In [2]:
filename = '/home/anaconda/var/grib/gfs.t06z.pgrb2.0p25.f000'
myfile = pygrib.open(filename)

In [20]:
# За координаты Москвы примем точку 55.75N,37.75E
# Данные хранятся в двумерных массивах. Индексы - узлы координатной сетки. Широта [90 .. -90], Долгота [0 ... 360 )
# Шаг - четверть градуса
# Идексы в массивах данных для Москвы:
MOSCOW_LAT_INDEX = (90 - 56)*4 + 1 
MOSCOW_LON_INDEX= 37*4  + 3

In [25]:
# Файл содержит GRIB-сообщения. 
# Каждое сообщение содержит значения параметра для всей координатной сетки
# У сообщения есть поля ID и name. 
# Сообщения с одинаковыми именами различаются высотой над уровнем моря, в этом случае select возвращает 
#  массив из нескольких элементов.
# Параметры, которые мы используем, имеют только одну высоту (2 метра или 10 метров), используем только первый эл-т массива.

moscow_t = myfile.select(name = '2 metre temperature')[0].data()[0][MOSCOW_LAT_INDEX][MOSCOW_LON_INDEX]
moscow_p = myfile.select(name = 'Pressure reduced to MSL')[0].data()[0][MOSCOW_LAT_INDEX][MOSCOW_LON_INDEX]
moscow_wg = myfile.select(name = 'Wind speed (gust)')[0].data()[0][MOSCOW_LAT_INDEX][MOSCOW_LON_INDEX]

# Для ветра есть "Восточная"- U и  "Северная"  - V составляющая.
moscow_wu = myfile.select(name = '10 metre U wind component')[0].data()[0][MOSCOW_LAT_INDEX][MOSCOW_LON_INDEX]
moscow_wv = myfile.select(name = '10 metre V wind component')[0].data()[0][MOSCOW_LAT_INDEX][MOSCOW_LON_INDEX]
moscow_w = (moscow_wu**2 + moscow_wv**2)**0.5

# Температура указывается в Кельвинах. Переведем в градусы Цельсия.
moscow_t = moscow_t - 273.15
print("Temperature: {:1.2f}; pressure: {:1.2f}, wind: {:1.2f}, gust to {:1.2f}".format(moscow_t, moscow_p, moscow_w,moscow_wg))

Temperature: 0.75; pressure: 101699.00, wind: 4.33, gust to 9.61


# Генерация текста по шаблонам

In [50]:
import pymorphy2

from utg import relations as r
from utg import logic
from utg import data
from utg import dictionary
from utg import words
from utg import templates
from utg import constructors

In [39]:
morph = pymorphy2.MorphAnalyzer()

def generate_forms(word):
    p = morph.parse(word)[0]
    res = []
    lexems = p.lexeme
    if 'NOUN' in p.tag:
        for i in range(len(lexems)):
            res.append(lexems[i][4][0][1])
        #print(res)
        if 'femn' in p.tag:
            res1 = res[:5]
            res1.append(res[6])
            res1.extend(res[7:])
            res1.extend(res1[6:])
            return(res1)
        else:
            res.extend(res[6:])
            return(res)
    if 'VERB' in p.tag or 'INFN' in p.tag:
        for i in [0,1,3,2,4]:
            res.append(lexems[i][4][0][1])
        return(res)
    if 'ADJF' in p.tag:
        for i in range(18):
            res.append(lexems[i][4][0][1])
        return(res)
    
a = generate_forms('Победить')
a

['победить', 'победил', 'победило', 'победила', 'победили']

# Flood

In [18]:
t = myfile.select(name = 'Wind speed (gust)')
t1 = t[0]

In [19]:
t1.latlons()

(array([[ 90.  ,  90.  ,  90.  , ...,  90.  ,  90.  ,  90.  ],
        [ 89.75,  89.75,  89.75, ...,  89.75,  89.75,  89.75],
        [ 89.5 ,  89.5 ,  89.5 , ...,  89.5 ,  89.5 ,  89.5 ],
        ..., 
        [-89.5 , -89.5 , -89.5 , ..., -89.5 , -89.5 , -89.5 ],
        [-89.75, -89.75, -89.75, ..., -89.75, -89.75, -89.75],
        [-90.  , -90.  , -90.  , ..., -90.  , -90.  , -90.  ]]),
 array([[  0.00000000e+00,   2.50000000e-01,   5.00000000e-01, ...,
           3.59250000e+02,   3.59500000e+02,   3.59750000e+02],
        [  0.00000000e+00,   2.50000000e-01,   5.00000000e-01, ...,
           3.59250000e+02,   3.59500000e+02,   3.59750000e+02],
        [  0.00000000e+00,   2.50000000e-01,   5.00000000e-01, ...,
           3.59250000e+02,   3.59500000e+02,   3.59750000e+02],
        ..., 
        [  0.00000000e+00,   2.50000000e-01,   5.00000000e-01, ...,
           3.59250000e+02,   3.59500000e+02,   3.59750000e+02],
        [  0.00000000e+00,   2.50000000e-01,   5.00000000e-01, .

In [75]:
# описываем существительное для словаря
coins_word = words.Word(type=r.WORD_TYPE.NOUN,
                        forms=[ u'монета', u'монеты', u'монете', u'монету', u'монетой', u'монете',    # единственнео число
                                u'монеты', u'монет', u'монетам', u'монеты', u'монетами', u'монетах',  # множественное число
                                u'монеты', u'монет', u'монетам', u'монеты', u'монетами', u'монетах'], # счётное число (заполнено для пример, может быть заполнено методом autofill_missed_forms)
                        properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.FEMININE)) # свойства: неодушевлённое, женский род

# описываем глагол для словаря
action_word = words.Word(type=r.WORD_TYPE.VERB,
                         # описываем только нужны нам формы слова (порядок важен и определён в utg.data.WORDS_CACHES[r.WORD_TYPE.VERB])
                         forms=[u'подарить', u'подарил', u'подарило', u'подарила', u'подарили'] + [u''] * (len(data.WORDS_CACHES[r.WORD_TYPE.VERB]) - 5),
                         properties=words.Properties(r.ASPECT.PERFECTIVE, r.VOICE.DIRECT) )
action_word.autofill_missed_forms() # заполняем пропущенные формы на основе введённых (выбираются наиболее близкие)

# создаём словарь для использования в шаблонах
test_dictionary = dictionary.Dictionary(words=[coins_word, action_word])

# создаём шаблон
template = templates.Template()

# externals — внешние переменные, не обязаны быть в словаре
template.parse(u'[Npc] [подарил|npc] [hero|дт] [coins] [монета|coins|вн].', externals=('hero', 'npc', 'coins'))

# описываем внешние переменные
hero = words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                                 forms=[u'герой', u'героя', u'герою', u'героя', u'героем', u'герое',
                                        u'герои', u'героев', u'героям', u'героев', u'героями', u'героях',
                                        u'герои', u'героев', u'героям', u'героев', u'героями', u'героях'],
                                 properties=words.Properties(r.ANIMALITY.ANIMATE, r.GENDER.MASCULINE)))

npc = words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                                forms=[u'русалка', u'русалки', u'русалке', u'русалку', u'русалкой', u'русалке',
                                       u'русалки', u'русалок', u'русалкам', u'русалок', u'русалками', u'русалках',
                                       u'русалки', u'русалок', u'русалкам', u'русалок', u'русалками', u'русалках'],
                                 properties=words.Properties(r.ANIMALITY.ANIMATE, r.GENDER.FEMININE)))

# осуществляем подстановку
result = template.substitute(externals={'hero': hero,
                                        'npc': npc,
                                        'coins': constructors.construct_integer(125)},
                             dictionary=test_dictionary)

result

'Русалка подарила герою 125 монет.'

In [36]:
result

'Русалка подарило герою 125 монет.'

In [80]:
def get_temp_phrase(temp: int):
    degree_word =  words.Word(type=r.WORD_TYPE.NOUN,
                                            forms=['градус', 'градуса', 'градусу', 'градус', 'градусом', 'градусе', 
                                                   'градусы', 'градусов', 'градусам', 'градусы', 'градусами', 'градусах', 
                                                   'градусы', 'градусов', 'градусам', 'градусы', 'градусами', 'градусах'],
                                            properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.MASCULINE))
    warm_word =  words.Word(type=r.WORD_TYPE.NOUN,
                                            forms=['тепло', 'тепла', 'теплу', 'тепло', 'теплом', 'тепле', 
                                                   '', '', '', '', '', '', 
                                                   '', '', '', '', '', ''],
                                            properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.MASCULINE))
    
    frost_word = words.Word(type=r.WORD_TYPE.NOUN,
                            forms=['мороз', 'мороза', 'морозу', 'мороза', 'морозом', 'морозе',  
                                   'морозы', 'морозов', 'морозам', 'морозов', 'морозами', 'морозах', 
                                   'морозы', 'морозов', 'морозам', 'морозов', 'морозами', 'морозах']

    #npc = words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
    #                            forms=[u'русалка', u'русалки', u'русалке', u'русалку', u'русалкой', u'русалке',
    #                                   u'русалки', u'русалок', u'русалкам', u'русалок', u'русалками', u'русалках',
    #                                   u'русалки', u'русалок', u'русалкам', u'русалок', u'русалками', u'русалках'],
    #                             properties=words.Properties(r.ANIMALITY.ANIMATE, r.GENDER.NEUTER)))
    
    my_dictionary = dictionary.Dictionary(words=[degree_word, warm_word])
    template = templates.Template()
    template.parse('Сейчас [temp] [градус|temp|вн] [тепло|рд].', externals=('temp'))
    result = template.substitute(externals={
                                           'temp': constructors.construct_integer(temp)
                                           }, dictionary=my_dictionary)
    return result
    
get_temp_phrase(-1)

'Сейчас -1 градус тепла.'

In [40]:
def generate_forms(word):
    p = morph.parse(word)[0]
    res = []
    lexems = p.lexeme
    if 'NOUN' in p.tag:
        for i in range(len(lexems)):
            res.append(lexems[i][4][0][1])
        #print(res)
        if 'femn' in p.tag:
            res1 = res[:5]
            res1.append(res[6])
            res1.extend(res[7:])
            res1.extend(res1[6:])
            return(res1)
        else:
            res.extend(res[6:])
            return(res)
    if 'VERB' in p.tag or 'INFN' in p.tag:
        for i in [0,1,3,2,4]:
            res.append(lexems[i][4][0][1])
        return(res)
    if 'ADJF' in p.tag:
        for i in range(18):
            res.append(lexems[i][4][0][1])
        return(res)

In [81]:
print(generate_forms('мороз'))

['мороз', 'мороза', 'морозу', 'мороза', 'морозом', 'морозе', 'мороз', 'мороз', 'мороз', 'мороз', 'мороз', 'мороз', 'морозы', 'морозов', 'морозам', 'морозов', 'морозами', 'морозах', 'мороз', 'мороз', 'мороз', 'мороз', 'мороз', 'мороз', 'морозы', 'морозов', 'морозам', 'морозов', 'морозами', 'морозах']
